In [1]:
# ==========================================================
# Artwork Analyzer v14 – Dynamic Indicator Edition
# ==========================================================
# Local-only (OpenCV + NumPy + Pillow). No network calls.
# Produces one annotated PNG with:
#   • On-image: ONLY color-coded visual indicators
#   • Below: legend + key metrics + paragraph curatorial summary
# Also saves curatorial_summary.txt and metrics.json
# ==========================================================

import os, json, time, textwrap, math
import numpy as np
import cv2
from PIL import Image
from math import atan2, degrees

# -----------------------------
# Utilities
# -----------------------------
def load_rgb(path):
    img = Image.open(path).convert("RGB")
    return np.array(img)

def ensure_dir(p): os.makedirs(p, exist_ok=True)

def to_uint8(img_f32):
    return np.clip(img_f32*255.0, 0, 255).astype(np.uint8)

def safe_gamma(img_f32):
    m = float(img_f32.mean()) if img_f32.size else 0.5
    if m < 0.20:
        img_f32 = np.power(img_f32, 0.6)
    elif m > 0.85:
        img_f32 = np.power(img_f32, 1.2)
    return np.clip(img_f32, 0, 1)

def resize_min_pixels(img, target_pixels=1_800_000):
    H, W = img.shape[:2]
    cur = H * W
    if cur >= target_pixels:
        return img, 1.0
    scale = math.sqrt(target_pixels / float(cur))
    newW, newH = int(W*scale), int(H*scale)
    new = cv2.resize(img, (newW, newH), interpolation=cv2.INTER_CUBIC)
    return new, scale

def wrap_text_lines(text, width_chars=90):
    return textwrap.wrap(text, width=width_chars)

# -----------------------------
# JSON-safe conversion
# -----------------------------
def make_json_safe(obj):
    if isinstance(obj, dict):
        return {k: make_json_safe(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [make_json_safe(x) for x in obj]
    elif isinstance(obj, (np.generic, np.number)):
        return obj.item()
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

# -----------------------------
# Legend bars (hue & orientation)
# -----------------------------
def make_hue_bar(hist, width=420, height=50):
    bar = np.zeros((height, width, 3), np.uint8)
    # gradient hue
    for x in range(width):
        h = int((x/width)*179)
        rgb = cv2.cvtColor(np.uint8([[[h, 200, 220]]]), cv2.COLOR_HSV2RGB)[0,0]
        bar[:, x] = rgb
    # overlay histogram
    if hist is not None and len(hist) > 0:
        bins = len(hist)
        bin_w = max(1, width // bins)
        norm = hist.astype(np.float32)
        if norm.max() > 0: norm /= (norm.max() + 1e-6)
        for i, v in enumerate(norm):
            x1 = i*bin_w; x2 = min(width-1, (i+1)*bin_w-1)
            h_px = int(v*(height-10))
            cv2.rectangle(bar, (x1, height-1), (x2, height-1-h_px), (0,0,0), -1)
    return bar

def make_orientation_bar(angles, energies, width=420, height=50):
    bar = np.full((height, width, 3), 240, np.uint8)
    if energies is None or len(energies)==0: return bar
    e = np.array(energies, dtype=np.float32)
    e = (e - e.min())/(e.max()-e.min()+1e-6)
    for a, v in zip(angles, e):
        x = int((a/180.0)*width)
        h_px = int(v*(height-10))
        cv2.rectangle(bar, (x-2, height-1), (x+2, height-1-h_px), (60,60,60), -1)
    cv2.putText(bar, "0°", (5,18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    cv2.putText(bar, "90°", (width//2-15,18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    cv2.putText(bar, "180°", (width-50,18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    return bar
# -----------------------------
# CIEDE2000 (pairwise for cluster centers)
# -----------------------------
def ciede2000(Lab1, Lab2):
    L1,a1,b1 = Lab1; L2,a2,b2 = Lab2
    C1 = np.sqrt(a1*a1 + b1*b1); C2 = np.sqrt(a2*a2 + b2*b2)
    avg_C = (C1 + C2)/2.0
    G = 0.5*(1 - np.sqrt((avg_C**7)/((avg_C**7)+25**7)))
    a1p, a2p = (1+G)*a1, (1+G)*a2
    C1p, C2p = np.sqrt(a1p*a1p + b1*b1), np.sqrt(a2p*a2p + b2*b2)
    h1p = (np.degrees(np.arctan2(b1, a1p)) + 360) % 360
    h2p = (np.degrees(np.arctan2(b2, a2p)) + 360) % 360
    dLp = L2-L1; dCp = C2p-C1p
    dhp = h2p-h1p
    if C1p*C2p == 0: dhp = 0
    else:
        if dhp > 180: dhp -= 360
        elif dhp < -180: dhp += 360
    dHp = 2*np.sqrt(C1p*C2p)*np.sin(np.radians(dhp)/2.0)
    avg_Lp = (L1+L2)/2.0
    if C1p*C2p == 0:
        avg_hp = h1p + h2p
    else:
        if abs(h1p-h2p) > 180:
            avg_hp = (h1p + h2p + 360)/2.0 if (h1p+h2p) < 360 else (h1p + h2p - 360)/2.0
        else:
            avg_hp = (h1p + h2p)/2.0
    T = (1 - 0.17*np.cos(np.radians(avg_hp-30)) +
         0.24*np.cos(np.radians(2*avg_hp)) +
         0.32*np.cos(np.radians(3*avg_hp+6)) -
         0.20*np.cos(np.radians(4*avg_hp-63)))
    d_ro = 30*np.exp(-((avg_hp-275)/25)**2)
    Rc = 2*np.sqrt(( ((avg_Cp := (C1p+C2p)/2.0)**7) / ((avg_Cp**7)+25**7) ))
    Sl = 1 + 0.015*(avg_Lp-50)**2 / np.sqrt(20 + (avg_Lp-50)**2)
    Sc = 1 + 0.045*avg_Cp
    Sh = 1 + 0.015*avg_Cp*T
    Rt = -np.sin(np.radians(2*d_ro))*Rc
    return float(np.sqrt((dLp/Sl)**2 + (dCp/Sc)**2 + (dHp/Sh)**2 + Rt*(dCp/Sc)*(dHp/Sh)))

def _kmeans_lab(lab_img, k, attempts=10):
    H, W = lab_img.shape[:2]
    Z = lab_img.reshape(-1,3).astype(np.float32)
    crit = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 80, 0.2)
    _r, labels, centers = cv2.kmeans(Z, k, None, crit, attempts, cv2.KMEANS_PP_CENTERS)
    labels2 = labels.reshape(H, W)
    return labels2, centers.astype(np.float32)

def _palette_quality(centers, entropy):
    if centers.shape[0] < 2:
        return entropy
    dEs = [ciede2000(centers[i], centers[j]) for i in range(len(centers)) for j in range(i+1, len(centers))]
    mean_dE = float(np.mean(dEs)) if dEs else 0.0
    spread = float(np.std(centers, axis=0).mean())
    return 0.6*(0.7*mean_dE + 0.3*spread) + 0.4*entropy, mean_dE

def analyze_color_deep_ensemble(img, ks=(6,8,10)):
    H, W = img.shape[:2]
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)

    # hue histogram for legend panel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hue = (hsv[:,:,0].astype(np.float32) * 2.0) % 360.0
    hue_hist, _ = np.histogram(hue, bins=36, range=(0,360))
    hue_hist = hue_hist.astype(np.float32)

    best = None
    best_score = -1
    for k in ks:
        labels2, centers = _kmeans_lab(lab, k)
        counts = np.bincount(labels2.reshape(-1), minlength=k).astype(np.float32)
        masks = [(labels2 == i).astype(np.uint8)*255 for i in range(k)]
        frac = counts/(H*W + 1e-6)
        p = frac[frac > 0]
        entropy = -float(np.sum(p*np.log2(p)))
        score, mean_dE = _palette_quality(centers, entropy)
        if score > best_score:
            best_score = score
            best = (k, labels2, centers, masks, counts, entropy, mean_dE)

    k_sel, labels2, centers, masks, counts, entropy, mean_dE = best
    rgb_centers = cv2.cvtColor(centers.reshape(-1,1,3).astype(np.uint8), cv2.COLOR_Lab2RGB).reshape(-1,3)
    palette = [tuple(map(int, c)) for c in rgb_centers]

    a, b = centers[:,1]-128, centers[:,2]-128
    hues_c = (np.degrees(np.arctan2(b, a)) + 360) % 360
    warm_ratio = float(((hues_c < 90) | (hues_c > 330)).sum()/len(hues_c))
    cool_ratio = float(((hues_c > 180) & (hues_c < 300)).sum()/len(hues_c))
    harmony = "mixed"
    if np.ptp(hues_c) < 35: harmony = "analogous"
    elif any(abs(hues_c[i]-hues_c[j])%360 > 150 for i in range(len(hues_c)) for j in range(i+1, len(hues_c))):
        harmony = "complementary"

    # ΔE proxy field from Lab gradients
    labf = lab.astype(np.float32)
    gxL = cv2.Sobel(labf[:,:,0], cv2.CV_32F, 1, 0); gyL = cv2.Sobel(labf[:,:,0], cv2.CV_32F, 0, 1)
    gxa = cv2.Sobel(labf[:,:,1], cv2.CV_32F, 1, 0); gya = cv2.Sobel(labf[:,:,1], cv2.CV_32F, 0, 1)
    gxb = cv2.Sobel(labf[:,:,2], cv2.CV_32F, 1, 0); gyb = cv2.Sobel(labf[:,:,2], cv2.CV_32F, 0, 1)
    grad_mag = np.sqrt((gxL**2 + gyL**2) + (gxa**2 + gya**2) + (gxb**2 + gyb**2))
    contrast_map = (grad_mag - grad_mag.min())/(grad_mag.max()-grad_mag.min()+1e-6)

    return dict(
        palette_rgb=palette,
        masks=masks,
        spatial_cohesion=float((counts/(H*W+1e-6)).max()),
        palette_entropy=float(entropy),
        mean_deltaE=float(mean_dE),
        max_deltaE=float(np.max([ciede2000(centers[i], centers[j]) for i in range(len(centers)) for j in range(i+1,len(centers))])) if len(centers)>1 else 0.0,
        warm_ratio=warm_ratio, cool_ratio=cool_ratio, harmony=harmony,
        contrast_map=contrast_map,
        hue_hist=hue_hist,
        k_selected=int(k_sel), ks=list(ks)
    )
# -----------------------------
# Texture: Gabor pyramid (scales 3..13, orientations 12)
# -----------------------------
def gabor_energy_pyramid(gray, scales=(3,5,7,9,11,13), orientations=12):
    H, W = gray.shape
    angles = np.linspace(0, 180, orientations, endpoint=False)
    combined = np.zeros((H,W), np.float32)
    angle_energy = np.zeros(orientations, np.float32)

    for oi, theta in enumerate(angles):
        theta_rad = np.deg2rad(theta)
        e_sum = np.zeros((H,W), np.float32)
        for k in scales:
            kernel = cv2.getGaborKernel((21,21), sigma=k, theta=theta_rad, lambd=10, gamma=0.5, psi=0)
            resp = cv2.filter2D(gray, cv2.CV_32F, kernel)
            e_sum += resp*resp
        combined = np.maximum(combined, e_sum)
        angle_energy[oi] = e_sum.mean()

    anisotropy = float((angle_energy.max() - angle_energy.min())/(angle_energy.mean()+1e-6))
    dom_angle = float(angles[int(np.argmax(angle_energy))])
    gmap = (combined - combined.min())/(combined.max()-combined.min()+1e-6)

    # simple entropy & roughness proxies
    hist, _ = np.histogram(gray, bins=256, range=(0,256))
    hist = hist.astype(np.float32); hist /= (hist.sum()+1e-6)
    entropy = -float(np.sum(hist*np.log2(hist + 1e-12)))
    roughness = float(gray.std())

    return dict(
        gabor_angles=angles, gabor_energy=angle_energy, gabor_map=gmap,
        gabor_anisotropy=anisotropy, gabor_dom_angle=dom_angle,
        lbp_entropy=entropy, lbp_roughness=roughness
    )

# -----------------------------
# Saliency: two-scale spectral residual + mask fusion
# -----------------------------
def spectral_saliency(grayf, blur=(9,9), sigma=2):
    fft = np.fft.fft2(grayf)
    loga = np.log(np.abs(fft)+1e-8)
    spec = np.exp((loga - cv2.blur(loga,(3,3))) + 1j*np.angle(fft))
    sal = np.abs(np.fft.ifft2(spec))**2
    sal = cv2.GaussianBlur(sal, blur, sigma)
    sal = (sal - sal.min())/(sal.max()-sal.min()+1e-6)
    return (sal*255).astype(np.uint8)

def subject_from_saliency_fused(s1, s2):
    H, W = s1.shape
    m1 = cv2.threshold(s1, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    m2 = cv2.threshold(s2, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    m = cv2.bitwise_and(m1, m2)
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, np.ones((7,7), np.uint8))
    cnts,_ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if cnts:
        x,y,w,h = cv2.boundingRect(max(cnts, key=cv2.contourArea))
    else:
        x,y,w,h = W//4, H//4, W//2, H//2
        cv2.rectangle(m, (x,y), (x+w,y+h), 255, -1)
    return dict(mask=m, bbox=(x,y,w,h))

def analyze_saliency_deep(img):
    g = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY).astype(np.float32)
    s_coarse = spectral_saliency(cv2.GaussianBlur(g,(0,0), 3.0), blur=(11,11), sigma=3)
    s_fine   = spectral_saliency(cv2.GaussianBlur(g,(0,0), 1.0), blur=(7,7),  sigma=1.5)
    fused = np.maximum(s_coarse, s_fine)
    sub = subject_from_saliency_fused(s_coarse, s_fine)
    Y, X = np.indices(fused.shape); w = fused.astype(np.float32)+1
    cx, cy = int((X*w).sum()/w.sum()), int((Y*w).sum()/w.sum())
    return dict(saliency_map=fused, focal=(cx,cy), subject_mask=sub["mask"], subject_bbox=sub["bbox"])

# -----------------------------
# Composition: multi-sweep Hough + vanishing point clustering
# -----------------------------
def line_intersection(p1, p2):
    (x1,y1,x2,y2), (x3,y3,x4,y4) = p1, p2
    den = (x1-x2)*(y3-y4) - (y1-y2)*(x3-x4)
    if abs(den) < 1e-6: return None
    px = ((x1*y2 - y1*x2)*(x3-x4) - (x1-x2)*(x3*y4 - y3*x4))/den
    py = ((x1*y2 - y1*x2)*(y3-y4) - (y1-y2)*(x3*y4 - y3*x4))/den
    return (px, py)

def cluster_points(points, grid):
    if not points: return None
    pts = np.array(points, dtype=np.float32)
    xs, ys = pts[:,0], pts[:,1]
    xbin = np.round(xs/grid).astype(int)
    ybin = np.round(ys/grid).astype(int)
    keys = xbin*100000 + ybin
    uniq, counts = np.unique(keys, return_counts=True)
    best_key = uniq[np.argmax(counts)]
    mask = (keys == best_key)
    bx = float(xs[mask].mean()); by = float(ys[mask].mean())
    return (bx, by)

def analyze_composition_deep(img, focal_xy):
    g = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    H, W = g.shape
    edges = cv2.Canny(g, 80, 200)

    lines = []
    params = [
        (1, np.pi/180, 100, min(H,W)//12, 15),
        (1, np.pi/180, 80,  min(H,W)//14, 20),
        (1, np.pi/180, 120, min(H,W)//10, 15)
    ]
    for rho, theta, thr, minL, gap in params:
        linesP = cv2.HoughLinesP(edges, rho, theta, thr, minLineLength=minL, maxLineGap=gap)
        if linesP is not None:
            for l in linesP:
                ln = tuple(map(int, l[0]))
                if math.hypot(ln[2]-ln[0], ln[3]-ln[1]) >= min(H,W)//14:
                    lines.append(ln)
    lines = lines[:300]

    # vanishing point
    inters = []
    for i in range(len(lines)):
        for j in range(i+1, len(lines)):
            ip = line_intersection(lines[i], lines[j])
            if ip is None: continue
            if -W*0.5 <= ip[0] <= W*1.5 and -H*0.5 <= ip[1] <= H*1.5:
                inters.append(ip)
    vp = cluster_points(inters, grid=max(20, min(H,W)//30))
    if vp is None: vp = (W/2, H/2)

    # thirds/golden deltas
    fx, fy = focal_xy
    thirds_pts = [(W/3,H/3),(2*W/3,H/3),(W/3,2*H/3),(2*W/3,2*H/3)]
    gfac = 0.618
    golden_pts = [(gfac*W,gfac*H),((1-gfac)*W,gfac*H),(gfac*W,(1-gfac)*H),((1-gfac)*W,(1-gfac)*H)]
    diag = np.hypot(W,H)
    thirds_delta = min(np.hypot(fx-x,fy-y) for (x,y) in thirds_pts)/(diag+1e-6)
    golden_delta = min(np.hypot(fx-x,fy-y) for (x,y) in golden_pts)/(diag+1e-6)

    # symmetry & edge balance
    g_f = g.astype(np.float32)/255.0
    left = g_f[:, :W//2]; right = np.fliplr(g_f[:, W - left.shape[1]:])
    mse = float(np.mean((left - right)**2))
    symmetry = float(1.0 - min(1.0, mse*4.0))
    lw, rw = float(np.sum(edges[:,:W//2])), float(np.sum(edges[:,W//2:]))
    balance = (rw - lw)/(lw + rw + 1e-6)

    return dict(
        lines=lines, vanishing_point=vp,
        thirds_delta=thirds_delta, golden_delta=golden_delta,
        symmetry=symmetry, edge_balance=balance
    )
# -----------------------------
# Illumination (direction + skew)
# -----------------------------
def illumination(img):
    L = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)[:,:,0].astype(np.float32)
    mu, sig = L.mean(), L.std()+1e-6
    skew = float((((L-mu)/sig)**3).mean())
    gx, gy = cv2.Sobel(L, cv2.CV_32F, 1,0), cv2.Sobel(L, cv2.CV_32F, 0,1)
    hi = (L > np.percentile(L, 90)).astype(np.uint8)
    if hi.sum() > 0:
        gx, gy = (gx*hi).sum(), (gy*hi).sum()
        ang = float((degrees(atan2(-gy, -gx)) + 360) % 360)
    else:
        ang = None
    return dict(lightness_skew=skew, light_direction_deg=ang)

# -----------------------------
# Curatorial summary (paragraph style, encouraging)
# -----------------------------
def generate_curatorial_summary(M):
    C, T, P, I = M["color"], M["texture"], M["composition"], M["illumination"]

    # color tone
    tone = {"analogous":"a gentle, harmonious range",
            "complementary":"a lively conversation of opposites"}.get(C["harmony"], "a balanced and nuanced palette")
    if C["mean_deltaE"] > 25: contrast = "confident contrasts that energize the field"
    elif C["mean_deltaE"] > 10: contrast = "measured contrasts that keep the eye engaged"
    else: contrast = "soft transitions that encourage a calm reading"

    # texture
    texture_phrase = (
        f"Texture carries a directional current (peaking near {T['gabor_dom_angle']:.0f}°), guiding attention across the surface"
        if T["gabor_anisotropy"] > 0.5 else
        "Texture remains even and poised, allowing color and figure to take the lead"
    )

    # composition
    if P["golden_delta"] < 0.12:
        comp_phrase = "The focal area whispers classical balance, close to golden proportion."
    elif P["thirds_delta"] < 0.12:
        comp_phrase = "The focal area falls near a rule-of-thirds node, feeling intuitive and open."
    else:
        comp_phrase = "The focal area resists fixed proportion, inviting free exploration in the field."

    # symmetry & balance
    sym_phrase = ("A notable symmetry steadies the composition."
                  if P["symmetry"] > 0.8 else
                  "Partial symmetry tempers the dynamism."
                  if P["symmetry"] > 0.5 else
                  "Asymmetry lends momentum and immediacy.")
    balance_phrase = ("Edge-weight remains steady."
                      if abs(P["edge_balance"]) < 0.1 else
                      "A deliberate imbalance animates the frame.")

    # illumination
    light_phrase = (f"Light arrives from roughly {int(I['light_direction_deg'])}°, gently shaping form."
                    if I["light_direction_deg"] is not None else
                    "Light diffuses evenly, allowing the whole surface to breathe.")
    mood = "bright and open" if I["lightness_skew"] > 0.5 else \
           "quietly contemplative" if I["lightness_skew"] < -0.5 else \
           "balanced and steady"

    paragraph = (
        f"This work invites a focused, unhurried look. The palette offers {tone}, with {contrast}. "
        f"{texture_phrase}. {comp_phrase} {sym_phrase} {balance_phrase} "
        f"{light_phrase} The overall mood feels {mood}, encouraging the eye to linger as color, gesture, and structure reinforce one another."
    )
    return paragraph
# -----------------------------
# Build colored indicator overlays (no text on artwork)
# -----------------------------
def build_overlays_only(img, M):
    """
    Returns RGB image with ONLY colored indicators over the artwork:
      - 🔴 focal rings
      - 🟣 ΔE proxy field
      - 🟠 texture energy
      - 🔵 composition lines (cyan/green/yellow)
      - 🟡 light direction arrow + vanishing point
      - (no captions on the artwork itself)
    """
    base = img.astype(np.float32)/255.0
    H, W = base.shape[:2]

    # 1) Texture (amber)
    gmap = M["texture"]["gabor_map"]
    gmap = cv2.GaussianBlur(gmap, (0,0), 1.0)
    gmap = (gmap - gmap.min())/(gmap.max()-gmap.min()+1e-6)
    tex_layer = np.dstack([gmap, gmap*0.45, 0*gmap]) * 0.35

    # 2) ΔE proxy (magenta)
    contrast = M["color"]["contrast_map"]
    contrast = cv2.GaussianBlur(contrast, (0,0), 1.0)
    contrast = (contrast - contrast.min())/(contrast.max()-contrast.min()+1e-6)
    de_layer = np.dstack([contrast*0.55, 0*contrast, contrast*0.55]) * 0.32

    # 3) Saliency (soft red wash)
    sal = M["saliency"]["saliency_map"].astype(np.float32)
    if sal.shape[:2] != (H, W):
        sal = cv2.resize(sal, (W, H), interpolation=cv2.INTER_CUBIC)
    sal = cv2.normalize(sal, None, 0, 1, cv2.NORM_MINMAX)
    sal_layer = np.dstack([sal*0.55, sal*0.25, sal*0.25]) * 0.25

    # 4) Lines (cyan/green/yellow by angle)
    line_layer = np.zeros_like(base)
    for (x1,y1,x2,y2) in M["composition"]["lines"][:240]:
        ang = (np.rad2deg(np.arctan2(y2-y1, x2-x1)) + 180) % 180
        col = (0,1,0) if 60<ang<120 else (0,0,1) if ang<30 or ang>150 else (1,1,0)
        cv2.line(line_layer,(x1,y1),(x2,y2),(col[2],col[1],col[0]),1,cv2.LINE_AA)
    line_layer = cv2.GaussianBlur(line_layer, (0,0), 2) * 0.50

    # Blend
    comp = base*0.90 + tex_layer + de_layer + sal_layer + line_layer
    comp = np.clip(comp, 0, 1)

    # 5) Subject contour + focal rings (magenta edge; red rings)
    comp_bgr = (comp[:,:,::-1]*255).astype(np.uint8).copy()
    mask = M["saliency"]["subject_mask"]
    cnts,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if cnts:
        cv2.drawContours(comp_bgr, cnts, -1, (255, 0, 255), 2)
    fx, fy = M["saliency"]["focal"]
    for r in range(6, 30, 6):
        cv2.circle(comp_bgr, (int(fx), int(fy)), r, (0,0,255), 1)

    # 6) Vanishing point + Light arrow
    vp = M["composition"].get("vanishing_point")
    if vp: cv2.circle(comp_bgr, tuple(map(int, vp)), 6, (0,255,255), -1)
    ang = M["illumination"].get("light_direction_deg")
    if ang is not None:
        Hc, Wc = comp_bgr.shape[:2]
        cx, cy = Wc-60, 60; r = min(Hc,Wc)//3
        ex, ey = int(cx - r*np.cos(np.radians(ang))), int(cy + r*np.sin(np.radians(ang)))
        cv2.arrowedLine(comp_bgr, (cx,cy), (ex,ey), (0,150,255), 3, tipLength=0.1)

    return comp_bgr[:,:,::-1]  # RGB uint8

# -----------------------------
# Build legend + captions + curatorial panel
# -----------------------------
def build_bottom_panel(M, width):
    """
    Returns a RGB uint8 panel containing:
      - Color swatch legend
      - Compact key metrics (one-liners)
      - Paragraph curatorial summary
      - Hue/Orientation bars on the right if space allows
    """
    f = cv2.FONT_HERSHEY_SIMPLEX
    line_h = 24
    pad = 16

    # Prepare text lines
    C, T, P, I = M["color"], M["texture"], M["composition"], M["illumination"]
    legend_title = "Legend"
    metrics_lines = [
        f"Color: harm={C['harmony']}  meanΔE={C['mean_deltaE']:.1f}  maxΔE={C['max_deltaE']:.1f}  entropy={C['palette_entropy']:.2f}  spatial={C['spatial_cohesion']:.2f}",
        f"Texture: anisotropy={T['gabor_anisotropy']:.2f}  dom={T['gabor_dom_angle']:.0f}°  rough={T['lbp_roughness']:.1f}",
        f"Composition: thirdsΔ={P['thirds_delta']:.3f}  goldenΔ={P['golden_delta']:.3f}  sym={P['symmetry']:.2f}  balance={P['edge_balance']:.2f}",
        f"Light: skew={I['lightness_skew']:.2f}  direction={I['light_direction_deg']}"
    ]
    paragraph = generate_curatorial_summary(M)
    para_lines = wrap_text_lines(paragraph, width_chars=max(60, min(120, width//12)))

    # Estimate panel height
    legend_rows = 2
    metrics_rows = len(metrics_lines)
    para_rows = len(para_lines)
    hue_bar_h = 44; ori_bar_h = 44; bars_block_h = hue_bar_h + 5 + ori_bar_h
    top_block = (legend_rows*line_h + pad*2)
    text_block = (metrics_rows*line_h + pad + para_rows*line_h + pad)
    panel_h = max(top_block + text_block, bars_block_h + pad*2) + 10

    panel = np.zeros((panel_h, width, 3), np.uint8)

    # Legend title
    y = pad + 4
    cv2.putText(panel, legend_title, (20, y+line_h), f, 0.9, (255,255,255), 2)

    # Swatch legend (row 1)
    y += line_h + 10
    sw = [
        ((20, y-20),(50, y+10),(0,0,255),   "Focal"),
        ((80, y-20),(110,y+10),(255,0,255), "Subject"),
        ((140,y-20),(170,y+10),(0,255,255), "Lines"),
        ((200,y-20),(230,y+10),(255,128,0), "Texture"),
        ((260,y-20),(290,y+10),(255,0,255), "ΔE"),
        ((320,y-20),(350,y+10),(255,255,255),"Thirds"),
        ((380,y-20),(410,y+10),(0,255,255), "Golden"),
        ((440,y-20),(470,y+10),(0,150,255), "Light→"),
        ((500,y-20),(530,y+10),(0,255,255), "Vanishing")
    ]
    for (x1,y1),(x2,y2), col, label in sw:
        if x2+120 < width:
            cv2.rectangle(panel, (x1,y1), (x2,y2), col, -1)
            cv2.putText(panel, label, (x2+8, y2), f, 0.6, (230,230,230), 1)

    # Key metrics
    y += line_h + pad
    for line in metrics_lines:
        cv2.putText(panel, line, (20, y), f, 0.6, (220,220,220), 1)
        y += line_h

    # Curatorial paragraph
    y += pad//2
    for pl in para_lines:
        cv2.putText(panel, pl, (20, y), f, 0.64, (235,235,235), 1)
        y += line_h

    # Right-side bars (hue & texture orientation), clipped to panel width
    hue_bar = make_hue_bar(M["color"]["hue_hist"], width=min(480, max(360, width//2)), height=hue_bar_h)
    ori_bar = make_orientation_bar(M["texture"]["gabor_angles"], M["texture"]["gabor_energy"], width=hue_bar.shape[1], height=ori_bar_h)
    hb_h, hb_w = hue_bar.shape[:2]; ob_h, ob_w = ori_bar.shape[:2]

    x0 = max(0, width - max(hb_w, ob_w) - 20)
    hue_bar_resized = hue_bar[:, :min(hb_w, width - x0)]
    ori_bar_resized = ori_bar[:, :min(ob_w, width - x0)]

    # place bars vertically centered in panel
    y_bars_top = max(pad, (panel_h - (hb_h + 5 + ob_h))//2)
    y1, y2 = y_bars_top, y_bars_top + hb_h
    if y2 <= panel.shape[0] and (x0 + hue_bar_resized.shape[1]) <= panel.shape[1]:
        panel[y1:y2, x0:x0+hue_bar_resized.shape[1]] = hue_bar_resized
    y1b, y2b = y_bars_top + hb_h + 5, y_bars_top + hb_h + 5 + ob_h
    if y2b <= panel.shape[0] and (x0 + ori_bar_resized.shape[1]) <= panel.shape[1]:
        panel[y1b:y2b, x0:x0+ori_bar_resized.shape[1]] = ori_bar_resized

    return panel
# -----------------------------
# Main pipeline (deep fidelity)
# -----------------------------
def analyze_artwork_high_fidelity(path, verbose=True, thorough=True):
    t0 = time.time()
    if verbose: print(f"🖼️ Loading image: {path}")
    img = load_rgb(path)
    H0, W0 = img.shape[:2]
    if verbose: print(f"   • Image size: {W0}×{H0}")

    # Upscale to ~1.8MP to ensure fidelity
    if thorough:
        img, scale = resize_min_pixels(img, target_pixels=1_800_000)
        H, W = img.shape[:2]
        if verbose and (H!=H0 or W!=W0):
            print(f"   • Rescaled for fidelity: {W}×{H}")
    else:
        H, W = H0, W0

    # COLOR
    if verbose: print("🎨 Color clustering (k=6,8,10)…", end="", flush=True)
    color = analyze_color_deep_ensemble(img, ks=(6,8,10))
    if verbose: print(f" ✓ (k*={color['k_selected']})")

    # TEXTURE
    if verbose: print("🪶 Texture (Gabor pyramid)…", end="", flush=True)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    texture = gabor_energy_pyramid(gray, scales=(3,5,7,9,11,13), orientations=12)
    if verbose: print(" ✓")

    # SALIENCY
    if verbose: print("🔥 Saliency (two-scale)…", end="", flush=True)
    sal = analyze_saliency_deep(img)
    if verbose: print(" ✓")

    # COMPOSITION
    if verbose: print("📐 Composition…", end="", flush=True)
    comp = analyze_composition_deep(img, sal["focal"])
    if verbose: print(" ✓")

    # ILLUMINATION
    if verbose: print("💡 Illumination…", end="", flush=True)
    illum = illumination(img)
    if verbose: print(" ✓")

    # COMPOSITE (overlays only on artwork)
    if verbose: print("✨ Building overlays…", end="", flush=True)
    M = dict(color=color, texture=texture, saliency=sal, composition=comp, illumination=illum)
    artwork_with_overlays = build_overlays_only(img, M)
    if verbose: print(" ✓")

    # PANEL (legend + metrics + paragraph)
    if verbose: print("🧭 Building panel…", end="", flush=True)
    panel = build_bottom_panel(M, width=artwork_with_overlays.shape[1])
    if verbose: print(" ✓")

    # STACK
    final = np.vstack([artwork_with_overlays, panel])
    final = to_uint8(safe_gamma(final.astype(np.float32)/255.0))

    # OUTPUTS
    out_dir = os.path.splitext(path)[0] + "_analysis"
    ensure_dir(out_dir)
    out_png = os.path.join(out_dir, "overlay_final_composite.png")
    cv2.imwrite(out_png, final)

    # Curatorial paragraph (also saved separately)
    paragraph = generate_curatorial_summary(M)
    out_txt = os.path.join(out_dir, "curatorial_summary.txt")
    with open(out_txt, "w", encoding="utf-8") as f:
        f.write(paragraph + "\n")

    # Metrics JSON
    metrics_py = make_json_safe(M)
    out_json = os.path.join(out_dir, "metrics.json")
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(metrics_py, f, indent=2)

    if verbose:
        print(f"\n✅ Deep analysis done in {time.time()-t0:.2f}s (thorough mode)")
        print(f"📁 Results saved to: {out_dir}")

# -----------------------------
# Run
# -----------------------------
if __name__ == "__main__":
    # ⬇️ Set your image path here:
    IMAGE_PATH = "/Users/alievanayasso/Documents/SlowMA/Raymond-Simboli-.jpg"
    analyze_artwork_high_fidelity(IMAGE_PATH, verbose=True, thorough=True)


🖼️ Loading image: /Users/alievanayasso/Documents/SlowMA/Raymond-Simboli-.jpg
   • Image size: 1140×797
   • Rescaled for fidelity: 1604×1121
🎨 Color clustering (k=6,8,10)… ✓ (k*=8)
🪶 Texture (Gabor pyramid)… ✓
🔥 Saliency (two-scale)… ✓
📐 Composition… ✓
💡 Illumination… ✓
✨ Building overlays… ✓
🧭 Building panel… ✓

✅ Deep analysis done in 18.43s (thorough mode)
📁 Results saved to: /Users/alievanayasso/Documents/SlowMA/Raymond-Simboli-_analysis
